In [ ]:
```xml
<VSCode.Cell language="markdown">
# API Gateway & Service Mesh

Infrastructure patterns for managing microservice communication: routing, security, observability, and traffic management.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    INFRASTRUCTURE LAYERS                                     │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   ┌─────────────────────────────────────────────────────────────────────┐   │
│   │                         CLIENTS                                      │   │
│   │                 (Web, Mobile, Third-party)                          │   │
│   └─────────────────────────────────┬───────────────────────────────────┘   │
│                                     │                                        │
│   ┌─────────────────────────────────▼───────────────────────────────────┐   │
│   │                       API GATEWAY                                    │   │
│   │   ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐  │   │
│   │   │  Auth    │ │  Rate    │ │  Route   │ │  Cache   │ │  Monitor │  │   │
│   │   │          │ │  Limit   │ │          │ │          │ │          │  │   │
│   │   └──────────┘ └──────────┘ └──────────┘ └──────────┘ └──────────┘  │   │
│   └─────────────────────────────────┬───────────────────────────────────┘   │
│                                     │                                        │
│   ┌─────────────────────────────────▼───────────────────────────────────┐   │
│   │                        SERVICE MESH                                  │   │
│   │                                                                      │   │
│   │  ┌────────────┐     ┌────────────┐     ┌────────────┐               │   │
│   │  │ ┌────────┐ │     │ ┌────────┐ │     │ ┌────────┐ │               │   │
│   │  │ │Sidecar │ │◀───▶│ │Sidecar │ │◀───▶│ │Sidecar │ │               │   │
│   │  │ └───┬────┘ │     │ └───┬────┘ │     │ └───┬────┘ │               │   │
│   │  │     │      │     │     │      │     │     │      │               │   │
│   │  │ ┌───▼────┐ │     │ ┌───▼────┐ │     │ ┌───▼────┐ │               │   │
│   │  │ │Service │ │     │ │Service │ │     │ │Service │ │               │   │
│   │  │ │   A    │ │     │ │   B    │ │     │ │   C    │ │               │   │
│   │  │ └────────┘ │     │ └────────┘ │     │ └────────┘ │               │   │
│   │  └────────────┘     └────────────┘     └────────────┘               │   │
│   │                                                                      │   │
│   └─────────────────────────────────────────────────────────────────────┘   │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## API Gateway Overview

```
API GATEWAY: Single entry point for all client requests

RESPONSIBILITIES:
┌─────────────────────┬─────────────────────────────────────────────────────┐
│ Request Routing     │ Route to appropriate backend service               │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Authentication      │ Validate tokens, API keys                          │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Authorization       │ Check permissions, scopes                          │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Rate Limiting       │ Throttle requests per client/endpoint              │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Load Balancing      │ Distribute traffic across instances                │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Caching             │ Cache responses to reduce backend load             │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Request Transform   │ Modify headers, body, query params                 │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Response Aggregation│ Combine data from multiple services                │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ SSL Termination     │ Handle HTTPS, decrypt at edge                      │
├─────────────────────┼─────────────────────────────────────────────────────┤
│ Observability       │ Logging, metrics, tracing                          │
└─────────────────────┴─────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## API Gateway Patterns

```
PATTERN 1: Simple Routing Gateway
─────────────────────────────────

  Client ──▶ Gateway ──▶ Service A
                    ──▶ Service B
                    ──▶ Service C

  - URL-based routing (/api/orders → Order Service)
  - Header-based routing (X-API-Version → specific version)


PATTERN 2: Backend for Frontend (BFF)
─────────────────────────────────────

  Web Client ──▶ Web BFF ─────┐
                              ├──▶ Services
  Mobile App ──▶ Mobile BFF ──┘

  - Optimized payloads per client
  - Different caching strategies
  - Client-specific authentication


PATTERN 3: Aggregating Gateway
──────────────────────────────

  Client ──▶ Gateway ──┬──▶ Order Service
                       ├──▶ User Service
                       └──▶ Product Service
                              │
                       ◀──────┴──── Composed Response

  - Single request, multiple backend calls
  - Response composition
  - Reduces client round trips
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Popular API Gateway Solutions

| Gateway | Type | Strengths | Best For |
|---------|------|-----------|----------|
| **Kong** | Open Source / Enterprise | Plugin ecosystem, Lua extensibility | Kubernetes, on-prem |
| **AWS API Gateway** | Managed | Lambda integration, WebSocket | AWS-native |
| **Azure API Management** | Managed | Developer portal, policies | Azure ecosystem |
| **Google Cloud Endpoints** | Managed | OpenAPI support, Cloud Run | GCP workloads |
| **NGINX** | Open Source | High performance, proven | High traffic |
| **Traefik** | Open Source | Auto-discovery, Let's Encrypt | Kubernetes, Docker |
| **Ambassador/Emissary** | Open Source | K8s native, Envoy-based | Kubernetes |
| **Apigee** | Enterprise | Analytics, monetization | Enterprise API programs |
</VSCode.Cell>
<VSCode.Cell language="python">
# API Gateway Implementation Example

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Callable, Any
from datetime import datetime, timedelta
from collections import defaultdict
import time
import hashlib
import json

# === RATE LIMITING ===
class SlidingWindowRateLimiter:
    """Token bucket rate limiter with sliding window."""
    
    def __init__(self, requests_per_minute: int = 100):
        self.limit = requests_per_minute
        self.window_seconds = 60
        self.requests: Dict[str, List[float]] = defaultdict(list)
    
    def is_allowed(self, client_id: str) -> tuple[bool, dict]:
        now = time.time()
        window_start = now - self.window_seconds
        
        # Clean old requests
        self.requests[client_id] = [
            t for t in self.requests[client_id] if t > window_start
        ]
        
        current_count = len(self.requests[client_id])
        remaining = max(0, self.limit - current_count)
        
        if current_count >= self.limit:
            retry_after = self.requests[client_id][0] - window_start
            return False, {
                "X-RateLimit-Limit": self.limit,
                "X-RateLimit-Remaining": 0,
                "Retry-After": int(retry_after)
            }
        
        self.requests[client_id].append(now)
        return True, {
            "X-RateLimit-Limit": self.limit,
            "X-RateLimit-Remaining": remaining - 1
        }

# === CACHING ===
class ResponseCache:
    """Simple in-memory cache with TTL."""
    
    def __init__(self, default_ttl: int = 60):
        self.cache: Dict[str, tuple[Any, float]] = {}
        self.default_ttl = default_ttl
    
    def _cache_key(self, method: str, path: str, query: str = "") -> str:
        key = f"{method}:{path}:{query}"
        return hashlib.md5(key.encode()).hexdigest()
    
    def get(self, method: str, path: str, query: str = "") -> Optional[Any]:
        key = self._cache_key(method, path, query)
        if key in self.cache:
            value, expires_at = self.cache[key]
            if time.time() < expires_at:
                return value
            del self.cache[key]
        return None
    
    def set(self, method: str, path: str, response: Any, 
            ttl: Optional[int] = None, query: str = ""):
        key = self._cache_key(method, path, query)
        expires_at = time.time() + (ttl or self.default_ttl)
        self.cache[key] = (response, expires_at)

# === ROUTING ===
@dataclass
class Route:
    path_prefix: str
    target_service: str
    methods: List[str] = field(default_factory=lambda: ["GET", "POST", "PUT", "DELETE"])
    strip_prefix: bool = True
    timeout: int = 30
    retry_count: int = 3
    cache_ttl: Optional[int] = None  # None = no caching

# === API GATEWAY ===
class APIGateway:
    """
    Simple API Gateway implementation demonstrating core patterns.
    """
    
    def __init__(self):
        self.routes: List[Route] = []
        self.rate_limiter = SlidingWindowRateLimiter(requests_per_minute=100)
        self.cache = ResponseCache()
        self.middleware: List[Callable] = []
        self.services: Dict[str, Callable] = {}  # Mock service registry
    
    def add_route(self, route: Route):
        self.routes.append(route)
        # Sort by path length (longest first for more specific matching)
        self.routes.sort(key=lambda r: len(r.path_prefix), reverse=True)
    
    def register_service(self, name: str, handler: Callable):
        """Register a mock service handler."""
        self.services[name] = handler
    
    def add_middleware(self, middleware: Callable):
        """Add request middleware (authentication, logging, etc.)."""
        self.middleware.append(middleware)
    
    def handle_request(self, request: dict) -> dict:
        """Main request handler."""
        
        client_id = request.get("client_id", "anonymous")
        method = request.get("method", "GET")
        path = request.get("path", "/")
        headers = request.get("headers", {})
        
        # 1. Rate Limiting
        allowed, rate_headers = self.rate_limiter.is_allowed(client_id)
        if not allowed:
            return {
                "status": 429,
                "body": {"error": "Too Many Requests"},
                "headers": rate_headers
            }
        
        # 2. Run middleware
        for mw in self.middleware:
            result = mw(request)
            if result:  # Middleware can short-circuit
                return result
        
        # 3. Find matching route
        route = self._match_route(path, method)
        if not route:
            return {"status": 404, "body": {"error": "Not Found"}}
        
        # 4. Check cache (for GET requests)
        if method == "GET" and route.cache_ttl:
            cached = self.cache.get(method, path)
            if cached:
                return {**cached, "headers": {**cached.get("headers", {}), "X-Cache": "HIT"}}
        
        # 5. Forward to service
        service_handler = self.services.get(route.target_service)
        if not service_handler:
            return {"status": 502, "body": {"error": "Service unavailable"}}
        
        # Transform path if stripping prefix
        service_path = path
        if route.strip_prefix:
            service_path = path[len(route.path_prefix):]
        
        response = service_handler({
            "method": method,
            "path": service_path,
            "headers": headers,
            "body": request.get("body")
        })
        
        # 6. Cache response
        if method == "GET" and route.cache_ttl and response.get("status") == 200:
            self.cache.set(method, path, response, route.cache_ttl)
        
        response["headers"] = {
            **response.get("headers", {}),
            **rate_headers,
            "X-Cache": "MISS"
        }
        return response
    
    def _match_route(self, path: str, method: str) -> Optional[Route]:
        for route in self.routes:
            if path.startswith(route.path_prefix) and method in route.methods:
                return route
        return None

# === DEMO ===
gateway = APIGateway()

# Add routes
gateway.add_route(Route("/api/orders", "order-service", cache_ttl=30))
gateway.add_route(Route("/api/users", "user-service", cache_ttl=60))
gateway.add_route(Route("/api/products", "product-service", cache_ttl=300))

# Register mock services
gateway.register_service("order-service", lambda r: {
    "status": 200, 
    "body": {"orders": [{"id": 1, "total": 99.99}]}
})
gateway.register_service("user-service", lambda r: {
    "status": 200, 
    "body": {"users": [{"id": 1, "name": "John"}]}
})

# Add authentication middleware
def auth_middleware(request: dict) -> Optional[dict]:
    token = request.get("headers", {}).get("Authorization")
    if request["path"].startswith("/api/") and not token:
        return {"status": 401, "body": {"error": "Unauthorized"}}
    return None

gateway.add_middleware(auth_middleware)

# Test requests
print("=== API Gateway Demo ===\n")

# Request without auth
response = gateway.handle_request({
    "client_id": "client-1",
    "method": "GET",
    "path": "/api/orders",
    "headers": {}
})
print(f"Without auth: {response['status']} - {response['body']}")

# Request with auth (first call - cache miss)
response = gateway.handle_request({
    "client_id": "client-1",
    "method": "GET",
    "path": "/api/orders",
    "headers": {"Authorization": "Bearer token123"}
})
print(f"With auth (1st): {response['status']} - Cache: {response['headers'].get('X-Cache')}")

# Same request (cache hit)
response = gateway.handle_request({
    "client_id": "client-1",
    "method": "GET",
    "path": "/api/orders",
    "headers": {"Authorization": "Bearer token123"}
})
print(f"With auth (2nd): {response['status']} - Cache: {response['headers'].get('X-Cache')}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Kong Gateway Configuration

```yaml
# kong.yml - Declarative Configuration

_format_version: "3.0"

services:
  # Order Service
  - name: order-service
    url: http://order-service:8080
    routes:
      - name: orders-route
        paths:
          - /api/orders
        methods:
          - GET
          - POST
          - PUT
        strip_path: true
    plugins:
      - name: rate-limiting
        config:
          minute: 100
          policy: local
      - name: jwt
        config:
          secret_is_base64: false
      - name: response-transformer
        config:
          add:
            headers:
              - "X-Gateway: Kong"

  # User Service
  - name: user-service
    url: http://user-service:8080
    routes:
      - name: users-route
        paths:
          - /api/users
    plugins:
      - name: proxy-cache
        config:
          content_type:
            - application/json
          cache_ttl: 300
          strategy: memory

# Global Plugins
plugins:
  - name: correlation-id
    config:
      header_name: X-Request-ID
      generator: uuid
  
  - name: prometheus
    config:
      per_consumer: true

  - name: request-termination
    service: order-service
    route: orders-route
    config:
      status_code: 503
      message: "Service temporarily unavailable"
    enabled: false  # Enable during maintenance

# Consumers (API clients)
consumers:
  - username: mobile-app
    keyauth_credentials:
      - key: mobile-api-key-123
    plugins:
      - name: rate-limiting
        config:
          minute: 1000  # Higher limit for mobile

  - username: third-party
    keyauth_credentials:
      - key: third-party-key-456
    plugins:
      - name: rate-limiting
        config:
          minute: 100

# Upstreams (load balancing)
upstreams:
  - name: order-service
    targets:
      - target: order-1:8080
        weight: 100
      - target: order-2:8080
        weight: 100
    healthchecks:
      active:
        http_path: /health
        interval: 5
        unhealthy:
          interval: 5
          http_failures: 3
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## AWS API Gateway Configuration

```yaml
# AWS SAM Template for API Gateway

AWSTemplateFormatVersion: '2010-09-09'
Transform: AWS::Serverless-2016-10-31

Globals:
  Api:
    Cors:
      AllowOrigin: "'*'"
      AllowHeaders: "'Content-Type,Authorization'"

Resources:
  # API Gateway
  OrderAPI:
    Type: AWS::Serverless::Api
    Properties:
      StageName: prod
      Auth:
        DefaultAuthorizer: CognitoAuthorizer
        Authorizers:
          CognitoAuthorizer:
            UserPoolArn: !GetAtt UserPool.Arn
      
      # Throttling
      MethodSettings:
        - ResourcePath: "/*"
          HttpMethod: "*"
          ThrottlingBurstLimit: 1000
          ThrottlingRateLimit: 500
      
      # Access logging
      AccessLogSetting:
        DestinationArn: !GetAtt ApiAccessLogs.Arn
        Format: '$requestId $httpMethod $resourcePath $status'

  # Lambda Integration
  GetOrdersFunction:
    Type: AWS::Serverless::Function
    Properties:
      Handler: orders.get_handler
      Runtime: python3.11
      Events:
        GetOrders:
          Type: Api
          Properties:
            RestApiId: !Ref OrderAPI
            Path: /orders
            Method: GET
        GetOrder:
          Type: Api
          Properties:
            RestApiId: !Ref OrderAPI
            Path: /orders/{orderId}
            Method: GET

  # Usage Plan for rate limiting per API key
  UsagePlan:
    Type: AWS::ApiGateway::UsagePlan
    Properties:
      UsagePlanName: StandardPlan
      Throttle:
        BurstLimit: 100
        RateLimit: 50
      Quota:
        Limit: 10000
        Period: MONTH
      ApiStages:
        - ApiId: !Ref OrderAPI
          Stage: prod

  # Cognito User Pool for auth
  UserPool:
    Type: AWS::Cognito::UserPool
    Properties:
      UserPoolName: OrderAPIUsers

  # CloudWatch Logs
  ApiAccessLogs:
    Type: AWS::Logs::LogGroup
    Properties:
      LogGroupName: /aws/apigateway/order-api
      RetentionInDays: 30

Outputs:
  ApiEndpoint:
    Value: !Sub "https://${OrderAPI}.execute-api.${AWS::Region}.amazonaws.com/prod"
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Service Mesh Overview

```
SERVICE MESH: Infrastructure layer for service-to-service communication

┌─────────────────────────────────────────────────────────────────────────────┐
│                        SERVICE MESH ARCHITECTURE                             │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   ┌─────────────────────────────────────────────────────────────────────┐   │
│   │                         CONTROL PLANE                                │   │
│   │   ┌──────────────┐  ┌──────────────┐  ┌──────────────┐              │   │
│   │   │   Config     │  │   Service    │  │  Certificate │              │   │
│   │   │   Server     │  │   Discovery  │  │   Authority  │              │   │
│   │   └──────────────┘  └──────────────┘  └──────────────┘              │   │
│   └─────────────────────────────────────────────────────────────────────┘   │
│                                     │                                        │
│                                     │ Push config                            │
│                                     ▼                                        │
│   ┌─────────────────────────────────────────────────────────────────────┐   │
│   │                          DATA PLANE                                  │   │
│   │                                                                      │   │
│   │   ┌─────────────────┐           ┌─────────────────┐                 │   │
│   │   │    Pod A        │           │    Pod B        │                 │   │
│   │   │  ┌───────────┐  │  mTLS     │  ┌───────────┐  │                 │   │
│   │   │  │  Sidecar  │◀─┼───────────┼─▶│  Sidecar  │  │                 │   │
│   │   │  │  (Envoy)  │  │           │  │  (Envoy)  │  │                 │   │
│   │   │  └─────┬─────┘  │           │  └─────┬─────┘  │                 │   │
│   │   │        │        │           │        │        │                 │   │
│   │   │  ┌─────▼─────┐  │           │  ┌─────▼─────┐  │                 │   │
│   │   │  │  Service  │  │           │  │  Service  │  │                 │   │
│   │   │  │     A     │  │           │  │     B     │  │                 │   │
│   │   │  └───────────┘  │           │  └───────────┘  │                 │   │
│   │   └─────────────────┘           └─────────────────┘                 │   │
│   └─────────────────────────────────────────────────────────────────────┘   │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘

SIDECAR RESPONSIBILITIES:
- Traffic management (routing, load balancing)
- Security (mTLS, authorization)
- Observability (metrics, logs, traces)
- Resilience (circuit breaker, retry, timeout)
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Service Mesh vs API Gateway

| Aspect | API Gateway | Service Mesh |
|--------|-------------|--------------|
| **Position** | Edge (North-South) | Internal (East-West) |
| **Traffic** | External → Internal | Service → Service |
| **Deployment** | Centralized | Distributed (sidecars) |
| **Auth Focus** | External clients | Service identities |
| **Use Case** | External API management | Internal traffic control |
| **Examples** | Kong, AWS API GW | Istio, Linkerd |

```
COMBINED ARCHITECTURE:

  External Traffic (North-South)
           │
           ▼
    ┌─────────────┐
    │ API Gateway │  ← External auth, rate limiting, routing
    └──────┬──────┘
           │
  ─────────┼───────── Internal Traffic (East-West)
           │
    ┌──────▼──────┐
    │   Service   │
    │    Mesh     │  ← mTLS, traffic splitting, observability
    │             │
    │  A ←─────▶ B│
    │  │         ││
    │  └───▶ C ◀─┘│
    └─────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Popular Service Mesh Solutions

| Mesh | Proxy | Strengths | Complexity |
|------|-------|-----------|------------|
| **Istio** | Envoy | Feature-rich, extensible | High |
| **Linkerd** | Linkerd-proxy (Rust) | Lightweight, simple | Low |
| **Consul Connect** | Envoy/Built-in | Multi-platform, service discovery | Medium |
| **AWS App Mesh** | Envoy | AWS integration | Medium |
| **Cilium** | eBPF (no proxy) | High performance | Medium |
| **Kuma** | Envoy | Multi-zone, simple | Medium |
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Istio Configuration Examples

```yaml
# VirtualService: Traffic routing rules

apiVersion: networking.istio.io/v1beta1
kind: VirtualService
metadata:
  name: order-service
spec:
  hosts:
    - order-service
  http:
    # Canary deployment: 90% v1, 10% v2
    - match:
        - headers:
            x-canary:
              exact: "true"
      route:
        - destination:
            host: order-service
            subset: v2
    - route:
        - destination:
            host: order-service
            subset: v1
          weight: 90
        - destination:
            host: order-service
            subset: v2
          weight: 10
    
    # Retry configuration
    retries:
      attempts: 3
      perTryTimeout: 2s
      retryOn: 5xx,reset,connect-failure
    
    # Timeout
    timeout: 10s

---
# DestinationRule: Load balancing and circuit breaker

apiVersion: networking.istio.io/v1beta1
kind: DestinationRule
metadata:
  name: order-service
spec:
  host: order-service
  trafficPolicy:
    connectionPool:
      tcp:
        maxConnections: 100
      http:
        h2UpgradePolicy: UPGRADE
        http1MaxPendingRequests: 100
        http2MaxRequests: 1000
    
    # Circuit breaker
    outlierDetection:
      consecutive5xxErrors: 5
      interval: 30s
      baseEjectionTime: 30s
      maxEjectionPercent: 50
    
    # Load balancing
    loadBalancer:
      simple: ROUND_ROBIN
  
  # Version subsets
  subsets:
    - name: v1
      labels:
        version: v1
    - name: v2
      labels:
        version: v2

---
# PeerAuthentication: mTLS configuration

apiVersion: security.istio.io/v1beta1
kind: PeerAuthentication
metadata:
  name: default
  namespace: production
spec:
  mtls:
    mode: STRICT  # Enforce mTLS for all services

---
# AuthorizationPolicy: Service-to-service auth

apiVersion: security.istio.io/v1beta1
kind: AuthorizationPolicy
metadata:
  name: order-service-policy
spec:
  selector:
    matchLabels:
      app: order-service
  rules:
    # Allow API Gateway
    - from:
        - source:
            principals: ["cluster.local/ns/default/sa/api-gateway"]
      to:
        - operation:
            methods: ["GET", "POST"]
            paths: ["/api/orders/*"]
    
    # Allow Payment Service for specific endpoint
    - from:
        - source:
            principals: ["cluster.local/ns/default/sa/payment-service"]
      to:
        - operation:
            methods: ["POST"]
            paths: ["/api/orders/*/confirm"]
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Linkerd Configuration Examples

```yaml
# ServiceProfile: Per-route metrics and retries

apiVersion: linkerd.io/v1alpha2
kind: ServiceProfile
metadata:
  name: order-service.production.svc.cluster.local
  namespace: production
spec:
  routes:
    - name: GET /api/orders
      condition:
        method: GET
        pathRegex: /api/orders
      responseClasses:
        - condition:
            status:
              min: 500
              max: 599
          isFailure: true
      timeout: 5s
      
    - name: POST /api/orders
      condition:
        method: POST
        pathRegex: /api/orders
      isRetryable: true
      timeout: 10s

---
# TrafficSplit: Canary deployment

apiVersion: split.smi-spec.io/v1alpha4
kind: TrafficSplit
metadata:
  name: order-service-split
spec:
  service: order-service
  backends:
    - service: order-service-v1
      weight: 900
    - service: order-service-v2
      weight: 100

---
# Server: Authorization policy

apiVersion: policy.linkerd.io/v1beta1
kind: Server
metadata:
  name: order-service
  namespace: production
spec:
  podSelector:
    matchLabels:
      app: order-service
  port: 8080
  proxyProtocol: HTTP/1

---
apiVersion: policy.linkerd.io/v1beta1
kind: ServerAuthorization
metadata:
  name: order-service-auth
  namespace: production
spec:
  server:
    name: order-service
  client:
    # Allow specific service accounts
    meshTLS:
      serviceAccounts:
        - name: api-gateway
        - name: payment-service
```
</VSCode.Cell>
<VSCode.Cell language="python">
# Sidecar Pattern Implementation (Conceptual)

from dataclasses import dataclass, field
from typing import Callable, Dict, Any, Optional
from datetime import datetime, timedelta
import random
import time
from enum import Enum
from functools import wraps

class CircuitState(Enum):
    CLOSED = "closed"
    OPEN = "open"
    HALF_OPEN = "half_open"

@dataclass
class CircuitBreaker:
    """Circuit breaker for the sidecar."""
    failure_threshold: int = 5
    reset_timeout: int = 30
    state: CircuitState = CircuitState.CLOSED
    failure_count: int = 0
    last_failure_time: Optional[float] = None
    
    def can_execute(self) -> bool:
        if self.state == CircuitState.CLOSED:
            return True
        if self.state == CircuitState.OPEN:
            if time.time() - self.last_failure_time >= self.reset_timeout:
                self.state = CircuitState.HALF_OPEN
                return True
            return False
        return True  # HALF_OPEN allows test request
    
    def record_success(self):
        self.failure_count = 0
        self.state = CircuitState.CLOSED
    
    def record_failure(self):
        self.failure_count += 1
        self.last_failure_time = time.time()
        if self.failure_count >= self.failure_threshold:
            self.state = CircuitState.OPEN

@dataclass
class SidecarProxy:
    """
    Sidecar proxy that intercepts service traffic.
    Implements: mTLS, circuit breaking, retry, metrics.
    """
    service_name: str
    circuit_breakers: Dict[str, CircuitBreaker] = field(default_factory=dict)
    request_count: int = 0
    error_count: int = 0
    latency_sum: float = 0.0
    
    def _get_circuit_breaker(self, target: str) -> CircuitBreaker:
        if target not in self.circuit_breakers:
            self.circuit_breakers[target] = CircuitBreaker()
        return self.circuit_breakers[target]
    
    def call_service(
        self,
        target_service: str,
        request: dict,
        actual_call: Callable,
        retries: int = 3,
        timeout: float = 5.0
    ) -> dict:
        """
        Proxy a call to another service with resilience patterns.
        """
        cb = self._get_circuit_breaker(target_service)
        
        # Check circuit breaker
        if not cb.can_execute():
            self.error_count += 1
            return {
                "status": 503,
                "error": f"Circuit open for {target_service}"
            }
        
        # Add tracing headers (mTLS would add certs in real impl)
        request["headers"] = request.get("headers", {})
        request["headers"]["x-request-id"] = f"req-{self.request_count}"
        request["headers"]["x-source-service"] = self.service_name
        request["headers"]["x-b3-traceid"] = f"trace-{random.randint(1000, 9999)}"
        
        # Retry logic with exponential backoff
        last_error = None
        for attempt in range(retries):
            start_time = time.time()
            self.request_count += 1
            
            try:
                response = actual_call(request)
                latency = time.time() - start_time
                self.latency_sum += latency
                
                if response.get("status", 200) < 500:
                    cb.record_success()
                    return response
                else:
                    raise Exception(f"Server error: {response.get('status')}")
                    
            except Exception as e:
                last_error = e
                cb.record_failure()
                self.error_count += 1
                
                if attempt < retries - 1:
                    backoff = (2 ** attempt) * 0.1
                    time.sleep(backoff)
        
        return {"status": 503, "error": str(last_error)}
    
    def get_metrics(self) -> dict:
        """Return Prometheus-style metrics."""
        avg_latency = self.latency_sum / max(1, self.request_count)
        return {
            f"sidecar_requests_total{{service=\"{self.service_name}\"}}": self.request_count,
            f"sidecar_errors_total{{service=\"{self.service_name}\"}}": self.error_count,
            f"sidecar_latency_avg{{service=\"{self.service_name}\"}}": round(avg_latency, 4),
            f"sidecar_circuit_state": {
                target: cb.state.value 
                for target, cb in self.circuit_breakers.items()
            }
        }

# === DEMO ===
print("=== Sidecar Proxy Demo ===\n")

# Create sidecar for Order Service
order_sidecar = SidecarProxy(service_name="order-service")

# Simulate calls to Payment Service
def mock_payment_call(request: dict) -> dict:
    # Simulate 30% failure rate
    if random.random() < 0.3:
        raise ConnectionError("Payment service unavailable")
    return {"status": 200, "body": {"payment_id": "pay-123"}}

# Make several calls
for i in range(10):
    response = order_sidecar.call_service(
        target_service="payment-service",
        request={"method": "POST", "path": "/payments"},
        actual_call=mock_payment_call,
        retries=2
    )
    status = response.get("status", "error")
    print(f"Call {i+1}: {status}")

print("\n=== Sidecar Metrics ===")
for metric, value in order_sidecar.get_metrics().items():
    print(f"  {metric}: {value}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Traffic Management Patterns

```
1. CANARY DEPLOYMENT
────────────────────
  
  Gradually shift traffic to new version:
  
  Day 1: v1 (99%) ─────────────▶ v2 (1%)
  Day 2: v1 (90%) ─────────────▶ v2 (10%)
  Day 3: v1 (50%) ─────────────▶ v2 (50%)
  Day 4: v1 (0%)  ─────────────▶ v2 (100%)


2. BLUE-GREEN DEPLOYMENT
────────────────────────

  ┌─────────────┐           ┌─────────────┐
  │  Blue (v1)  │◀──100%    │  Green (v2) │◀──0%
  │   Active    │           │   Standby   │
  └─────────────┘           └─────────────┘
         │                         │
         └──── Instant switch ─────┘
                     ▼
  ┌─────────────┐           ┌─────────────┐
  │  Blue (v1)  │◀──0%      │  Green (v2) │◀──100%
  │   Standby   │           │   Active    │
  └─────────────┘           └─────────────┘


3. A/B TESTING
──────────────

  Route by user attribute:
  
  if user.segment == "beta_testers":
      route_to(v2)
  else:
      route_to(v1)


4. HEADER-BASED ROUTING
───────────────────────

  X-Debug: true        → Debug service
  X-Version: v2        → Version 2
  X-Environment: test  → Test environment
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Observability in Service Mesh

```
THREE PILLARS OF OBSERVABILITY:

┌─────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│   METRICS                    LOGGING                   TRACING              │
│   ───────                    ───────                   ───────              │
│                                                                              │
│   Quantitative data          Event records             Request flow         │
│   Aggregated over time       Per-request details       Across services      │
│                                                                              │
│   ┌─────────────────┐        ┌─────────────────┐      ┌─────────────────┐   │
│   │ Prometheus      │        │ ELK Stack       │      │ Jaeger          │   │
│   │ • request_count │        │ • Log entries   │      │ • Spans         │   │
│   │ • error_rate    │        │ • Search/filter │      │ • Dependencies  │   │
│   │ • latency_p99   │        │ • Dashboards    │      │ • Bottlenecks   │   │
│   └─────────────────┘        └─────────────────┘      └─────────────────┘   │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘


DISTRIBUTED TRACING EXAMPLE:

  Request: GET /api/orders/123

  ┌──────────────────────────────────────────────────────────────────────────┐
  │ Trace ID: abc123                                                          │
  ├──────────────────────────────────────────────────────────────────────────┤
  │                                                                           │
  │  [API Gateway]          0ms ████████████████████████████████████ 150ms   │
  │   └─[Order Service]    20ms ████████████████████████████ 130ms           │
  │      ├─[User Service]  30ms ████████ 60ms                                │
  │      └─[Product Svc]   65ms ████████████ 110ms                           │
  │                                                                           │
  └──────────────────────────────────────────────────────────────────────────┘

  Each span contains:
  - Service name
  - Operation name  
  - Start time, duration
  - Tags (http.status, error, etc.)
  - Logs (events during span)
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Decision Framework: Gateway vs Mesh vs Both

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                      DECISION MATRIX                                         │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  NEED                           │  SOLUTION                                 │
│  ────                           │  ────────                                 │
│                                 │                                           │
│  External API management        │  API Gateway                              │
│  Rate limiting by API key       │  API Gateway                              │
│  Developer portal               │  API Gateway (Kong, Apigee)               │
│  API monetization               │  API Gateway (Apigee)                     │
│                                 │                                           │
│  mTLS between services          │  Service Mesh                             │
│  Canary deployments             │  Service Mesh (or Argo Rollouts)          │
│  Circuit breaking               │  Service Mesh or App-level                │
│  Distributed tracing            │  Service Mesh (or manual instrumentation) │
│  Service-to-service auth        │  Service Mesh                             │
│                                 │                                           │
│  Both external + internal needs │  API Gateway + Service Mesh               │
│                                 │                                           │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  COMPLEXITY BUDGET:                                                         │
│                                                                              │
│  Small team (<10), few services     → API Gateway only                      │
│  Medium team, 10-30 services        → API Gateway + consider mesh           │
│  Large org, 50+ services            → API Gateway + Service Mesh            │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Summary: Key Takeaways

| Component | Purpose | When to Use |
|-----------|---------|-------------|
| **API Gateway** | Edge traffic management | Always for external APIs |
| **Service Mesh** | Internal traffic management | Large-scale microservices |
| **Sidecar Proxy** | Per-service infrastructure | When using service mesh |
| **BFF** | Client-specific aggregation | Multiple client types |
| **mTLS** | Zero-trust security | Production microservices |
| **Circuit Breaker** | Failure isolation | Any distributed system |
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Further Reading

- **Topics in this Knowledge Base:**
  - [Service Communication](./02_service_communication.ipynb)
  - [Distributed Patterns](./04_distributed_patterns.ipynb)
  - [Microservices Overview](./00_overview.ipynb)

- **External Resources:**
  - [Istio Documentation](https://istio.io/docs/)
  - [Linkerd Documentation](https://linkerd.io/docs/)
  - [Kong Gateway](https://docs.konghq.com/)
  - [Envoy Proxy](https://www.envoyproxy.io/docs/)
</VSCode.Cell>
```